<img src='http://www.u.arizona.edu/~hammond/hltlogo1.png' style="float:right">

Linguistics 578<br>
Fall 2022<br>
Hammond

## Things to remember about any homework assignment:

1. For this assignment, you will edit this jupyter notebook and turn it in. Do not turn in pdf files or separate `.py` files.
1. Late work is not accepted.
1. Given the way I grade, you should try to answer *every* question, even if you don't like your answer or have to guess.
1. You may *not* use `python` modules that we have not already used in class.
1. You may certainly talk to your classmates about the assignment, but everybody must turn in *their own* work. It is not acceptable to turn in work that is essentially the same as the work of classmates.
1. All code must run. It doesn't have to be perfect, it may not do all that you want it to do, but it must run without error.
1. Code must run in reasonable time. Assume that if it takes more than *5 minutes* to run (on your machine), that's too long.
1. Please do not add, remove, or copy autograded cells.
1. Make sure to select `restart, run all cells` from the `kernel` menu when you're done and before you turn this in!

***

***my name***: Rachel the magnificent speech tech genius Hansen (rtmstgh)

***people I talked to about the assignment***: The girl code squad

***

## Homework #5

Here are the imports. Please do not import anything else.

In [1]:
import numpy as np

1. The book includes an implementation of HMMs, but let's do a different implementation here. The one here will be quite spare. We have a class that holds a vector for $\pi$, a transition matrix, an emission matrix, and symbol lookup tables. The bare bones for these are given below. Fill in the `check()` method that checks that these are well structured and work together properly. (Keep in mind that that this has to run on other machines where there may be miniscule differences in floating point values.)

In [2]:
class HMM:
    def add_symbols(self,symbols):
        self.s2i = {symbol:i for i,symbol in enumerate(symbols)}
        self.i2s = {i:symbol for i,symbol in enumerate(symbols)}
    def add_start(self,pi):
        self.pi = pi
    def add_emissions(self,em):
        self.em = em
    def add_transitions(self,tm):
        self.tm = tm
    def check(self):
        '''checks that the data structures are present
        and all correct in terms of shapes and values
        
        returns:
            boolean: true if all tests pass
        '''
        #testing that pi variables exist
        try: self.pi
        except:
            return False
        
        #testing that em variables exist
        try: self.em
        except:
            return False
        
        #testing that tm variables exist
        try: self.tm
        except:
            return False
        
        #comparing length of pi to tm dimensions ex: if len of pi= 2, shape of tm = (2,2)
        if len(self.pi) == len(self.tm):
            if len(self.pi) == len(self.tm[1]):
                
                #testing all probabilities add to 1
                #pi values
                if sum(self.pi) != 1:
                    print ('error')
                    return False
                #em values
                addem = np.sum(self.em,axis=1)
                for i in addem:
                    if i != 1:
                        return False
                #tm values
                addtm = np.sum(self.tm,axis=1)
                for i in addtm:
                    if i != 1:
                        return False
                return True 
                
            else:
                return False
        else:
            return False

            
       # raise NotImplementedError()

In [3]:
hmm1 = HMM()
hmm1.add_symbols('abc')
assert not hmm1.check()

In [4]:
hmm2 = HMM()
hmm2.add_symbols('abc')
hmm2.add_start(np.array([.6,.4]))
hmm2.add_emissions(np.array([
    [.3,.1,.6],
    [.2,.5,.3]
]))
hmm2.add_transitions(np.array([
    [.2,.8],
    [.6,.4]
]))
assert hmm2.check()

In [5]:
hmm3 = HMM()
hmm3.add_symbols('abc')
hmm3.add_start(np.array([.2,.8]))
hmm3.add_emissions(np.array([
    [.3,.1,.6],
    [.2,.5,.3]
]))
hmm3.add_transitions(np.array([
    [.1,.1,.8],
    [.3,.3,.4],
    [.7,.2,.1]
]))
assert not hmm3.check()

In [6]:
hmm4 = HMM()
hmm4.add_symbols('abc')
hmm4.add_start(np.array([.2,.8]))
hmm4.add_emissions(np.array([
    [.3,.1,.6],
    [.2,.5,.3]
]))
hmm4.add_transitions(np.array([
    [.1,.8],
    [.7,.3]
]))
assert not hmm4.check()

2. Let's now improve the implementation by adding a method to calculate forward probabilities. We'll do this by inheriting from the original implementation above.

In [7]:
class HMMbetter(HMM):
    def forward(self,s):
        '''calculate forward probabilities
        
        args:
            s: a string
        returns:
            the probability of the string
            the forward probability grid
        '''
        nodecount = self.tm.shape [0]
        # make matrix
        self.grid = np.zeros([nodecount,len(s)+1])
        #go through the input letter by letter
        for letpos in range(len(s)):
            # get letter index
            letidx = self.s2i[s[letpos]]
            # get probs to the left
            #if we ✬re in the first col , use start probs
            if letpos == 0:
                prev = self.pi
            else :
                prev = self.grid[:,letpos]
            #go row by row , filling in values
            # sum for all states
            for node in range(nodecount):
                val = 0
                if letpos == 0:
                    val = prev[node]
                    self.grid [node,letpos] = val
                    val = prev[node] * self.em[node,letidx]
                else:
                    for n2 in range(nodecount):
                        curval = prev[n2]
                        curval *= self.tm[n2,node]
                        curval *= self.em[node,letidx]
                        val += curval
                self.grid [node , letpos+1 ] = val
        return self.grid [: , -1]. sum ( axis =0) , self.grid
 #       raise NotImplementedError()

In [8]:
hmm5 = HMMbetter()
hmm5.add_symbols('abc')
hmm5.add_start(np.array([.2,.8]))
hmm5.add_emissions(np.array([
    [.3,.1,.6],
    [.2,.5,.3]
]))
hmm5.add_transitions(np.array([
    [.8,.2],
    [.7,.3]
]))
p,g = hmm5.forward('acc')
assert np.isclose(p,.061,atol=.001)

In [9]:
assert g.shape == (2,4)

In [10]:
assert np.isclose(g,[
    [.2,.06,.096,.05364],
    [.8,.16,.018,.00738]
],atol=.001).all()

3. Finally, let's implement the Viterbi algorithm. The method should return four things:
    1. the most likely state sequence
    1. the probability of that sequence
    1. the grid with probability values
    1. a separate grid with previous state

  You should be able to reuse/retask a lot of code from the previous question.
  
  (Note the orientation and shape of the various matrices in the tests below; these may differ from what you expect.)

In [11]:
class HMMbest(HMMbetter):
    def viterbi(self,s):
        '''viterbi algorithm
        args:
            s: a string
        returns:
            the most likely path (as a list of ints)
            probability of that path
            accumulated probability matrix
            accumulated path matrix
        '''
        nodecount = self.tm.shape [0]
        # make matrix
        grid = np.zeros([nodecount,len(s)+1])
        best = np.zeros ([nodecount, (len(s))],dtype=int)
        sepgrid = np.zeros([nodecount, len(s)-1])
        #go through the input letter by letter
        for letpos in range(len(s)):
            # get letter index
            letidx = self.s2i[s[letpos]]
            # get probs to the left
            #if we ✬re in the first col , use start probs
            if letpos == 0:
                prev = self.pi
            else :
                prev = grid[:,letpos]
            #go row by row , filling in values
            # sum for all states
            for node in range(nodecount):
                val = []
                nodez = {}
                if letpos == 0:
                    grid[node,letpos] = prev[node]
                    grid[node,letpos+1] = prev[node]*self.em[node,letidx]
                    best[node,letpos] = node
                else:
                    for n2 in range(nodecount):
                        curval = prev[n2]
                        curval *= self.tm[n2,node]
                        curval *= self.em[node,letidx]
                        val.append (curval)
                        nodez[curval] = n2
                    #keeps track of the max value
            #        print (nodez)
                    grid [node , letpos+1 ] = max(val)
                    #keeps track of where that max came from
                    best [node, letpos] = val.index(max(val))
                    sepgrid[node, letpos-1] = nodez.get(max(val))
         #reads back the best path
        last = grid.argmax(axis=0)[-1]
        res = [last]
        i = len(s) -1
        while i >= 1:
            res.append(best[res[-1],i])
            i -= 1
        print(sepgrid)
        return res, grid [: , -1]. max() , grid, sepgrid

       # raise NotImplementedError

In [12]:
hmm6 = HMMbest()
hmm6.add_symbols('abc')
hmm6.add_start(np.array([.2,.8]))
hmm6.add_emissions(np.array([
    [.3,.1,.6],
    [.2,.5,.3]
]))
hmm6.add_transitions(np.array([
    [.8,.2],
    [.7,.3]
]))
path,prob,probs,paths = hmm6.viterbi('acc')
assert path == [0,0,1]

[[1. 0.]
 [1. 0.]]


In [13]:
assert np.isclose(prob,.03225,atol=0.001)

In [14]:
assert probs.shape == (2,4)

In [15]:
assert np.isclose(probs,np.array([
    [0.2,0.06,0.0672,0.032256],
    [0.8,0.16,0.0144,0.004032]
]),atol=.01).all()

In [16]:
assert paths.shape == (2,2)